In [1]:
import os
import huggingface_hub
from dotenv import load_dotenv

load_dotenv("../keys.env")

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
import json
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain_openai import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_huggingface.llms import HuggingFacePipeline

In [3]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
    
def convert_to_dialogue_format(messages):
    dialogue = ""
    for msg in messages:
        role = msg['role']
        content = msg['content']
        if role == 'user':
            dialogue += f"user: {content}\n"
        elif role == 'assistant':
            dialogue += f"assistant: {content}\n"
    return dialogue

In [4]:
eval_data = load_jsonl("../dataset/eval.jsonl")
print(len(eval_data))
print(eval_data[0].keys())

220
dict_keys(['eval_id', 'msg'])


In [5]:
sample = eval_data[8]['msg']
print(sample)

diag1 = convert_to_dialogue_format(sample)
print(diag1)

[{'role': 'user', 'content': '이란 콘트라 사건이 뭐야'}, {'role': 'assistant', 'content': '이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다.'}, {'role': 'user', 'content': '이 사건이 미국 정치에 미친 영향은?'}]
user: 이란 콘트라 사건이 뭐야
assistant: 이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다.
user: 이 사건이 미국 정치에 미친 영향은?



In [6]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

if tokenizer.pad_token is None:
    if tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token
    else:
        tokenizer.pad_token = tokenizer.eos_token = '<|endoftext|>' 

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct.
403 Client Error. (Request ID: Root=1-66fe1c61-14935228424b13c81b04cd8b;c9101a63-5ba5-421a-adf9-8ff0e0a212f8)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-3B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct to ask for access.

In [24]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=20,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    clean_up_tokenization_spaces=True  # 또는 False로 설정 가능
)

hf = HuggingFacePipeline(pipeline=pipe)

In [25]:
# class CustomOutputParser(BaseOutputParser):
#     def parse(self, text: str) -> dict:
#         split_text = text.split("출력:", 1)
#         if len(split_text) > 1:
#             return split_text[1].strip()
#         else:
#             return text

class CustomOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # 모델이 출력한 텍스트에서 JSON 형식을 찾고 파싱
        try:
            # "출력:" 이후의 부분만 추출
            split_text = text.split("출력:", 1)
            if len(split_text) > 1:
                json_str = split_text[1].strip()  # 불필요한 공백 제거
                json_output = json.loads(json_str)
                return json_output
            else:
                return {}  # 출력 형식이 맞지 않으면 빈 딕셔너리 반환
        except json.JSONDecodeError:
            return {}  # JSON 파싱 실패 시 빈 딕셔너리 반환

# template = """주어진 입력이 user와 assistant가 나눈 멀티턴 대화인지 분류해주세요. 답변은 오직 json 형식으로 반환합니다. 추가적인 답변은 생성하지마세요.

# 입력: {query}
# 출력예시: {{"multi-turn": true}} 또는 {{"multi-turn": false}}
# 출력:"""

template = """
주어진 입력이 user와 assistant가 나눈 멀티턴 대화인지 분류해주세요. 반드시 아래 형식을 따르세요. 다른 어떤 텍스트도 포함하지 마세요.

출력 형식:
{{
  "multi-turn": true
}} 또는 {{
  "multi-turn": false
}}

입력 예시:
user: 이란 콘트라 사건이 뭐야
assistant: 이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다.
user: 이 사건이 미국 정치에 미친 영향은?

출력 예시:
{{
  "multi-turn": true
}}

입력: {query}
출력:
"""

rag_prompt = PromptTemplate.from_template(template)
output_parser = CustomOutputParser()
rag_chain = {"query": RunnablePassthrough()} | rag_prompt | hf | output_parser


In [ ]:
response = rag_chain.invoke(diag1)
print(response)